In [1]:
import re
import sys
import json

import pandas as pd
import numpy as np

import repetition_matrix

import seaborn as sns

In [2]:
from lyricsgenius import Genius

genius = Genius(
    'in424DYZRH1oa--Iyzce_Ue6uLYNqt1Q6-Yq20PUSYl-PjvMNuuaX0YBiwcSFt9_',
    retries=50
)

In [3]:
song_parts_regex = r"\[Intro.*?\]|\[Verse [0-9].*?\]|\[Refrain.*?\]|\[Pre-Chorus.*?\]|\[Chorus.*?\]|\[Post-Chorus.*?\]|\[Hooks.*?\]|\[Riffs/Basslines.*?\]|\[Scratches.*?\]|\[Bridge.*?\]|\[Interlude.*?\]|\[Break.*?\]|\[Skit.*?\]|\[Collision.*?\]|\[Instrumental or Solo.*?\]|\[Ad lib.*?\]|\[Segue.*?\]|\[Outro.*?\]"

In [22]:
def get_chorus(song_object):
    raw_lyrics_list = song_object['lyrics'].split('\n')
    filtered_lyrics_list = filter(lambda line: line != '', raw_lyrics_list)
    chorus = []
    chorus_indicator = False
    for line in filtered_lyrics_list:
        if chorus_indicator:
            if re.match(song_parts_regex, line):
                break
            chorus.append(line)
        if re.match(r"\[Chorus.*?\]", line):
            chorus_indicator = True
    return chorus

In [23]:
# def load_jsons():
#     song_objects = []
#     for i in range(0, 23500, 100):
#         with open(f'test_data/song_jsons_{i}_to_{i+100}.json') as json_file:
#             data = json.load(json_file)
#             song_objects += data
#     return song_objects

In [36]:
with open(f'data/all_songs.json') as json_file:
    song_objects = json.load(json_file)
    print(song_objects['0'])
    song_objects = [song_objects[index] for index in song_objects]

{'song': 'Blinding Lights by The Weeknd', 'artist': 'The Weeknd', 'chorus': ["I said, ooh, I'm blinded by the lights", "No, I can't sleep until I feel your touch", "I said, ooh, I'm drowning in the night", "Oh, when I'm like this, you're the one I trust", 'Hey, hey, hey']}


In [40]:
with open(f'data/all_songs.json', 'x') as json_file:
    json.dump(song_objects, json_file)

In [73]:
songs = list(filter(lambda song: len(song['chorus']) >= 4, song_objects))
songs

[{'song': 'Blinding Lights by The Weeknd',
  'artist': 'The Weeknd',
  'chorus': ["I said, ooh, I'm blinded by the lights",
   "No, I can't sleep until I feel your touch",
   "I said, ooh, I'm drowning in the night",
   "Oh, when I'm like this, you're the one I trust",
   'Hey, hey, hey'],
  'repetitiveness': 0.26507532447576526},
 {'song': 'Radioactive by Imagine Dragons',
  'artist': 'Imagine Dragons',
  'chorus': ['Whoa-oh, whoa',
   "I'm radioactive, radioactive",
   'Whoa-oh, whoa',
   "I'm radioactive, radioactive"],
  'repetitiveness': 0.4640672504901885},
 {'song': 'Sail by AWOLNATION',
  'artist': 'AWOLNATION',
  'chorus': ['Sail', 'Sail', 'Sail', 'Sail', 'Sail'],
  'repetitiveness': 1.0000000000000002},
 {'song': 'How Do I Live by LeAnn Rimes',
  'artist': 'LeAnn Rimes',
  'chorus': ['How do I live without you? I want to know',
   'How do I breathe without you if you ever go?',
   'How do I ever, ever survive?',
   'How do I, how do I, oh, how do I live?'],
  'repetitiveness'

In [74]:
df = pd.DataFrame(songs)

In [43]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

2022-12-05 13:51:21.535751: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-05 13:51:21.536057: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-12-05 13:51:24.429963: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-05 13:51:24.449831: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [63]:
def calculate_similarity_score(chorus):
    chorus_embeddings = embed(chorus)
    corr = np.inner(chorus_embeddings, chorus_embeddings).tolist()

    indices = range(len(corr))
    for row in indices:
        for col in indices:
            if corr[row][col] > 0.99:
                corr[row][col] = 1
            if corr[row][col] < 0:
                corr[row][col] = 0

    count_not_identity = len(corr) ** 2 - len(corr)
    avg_similarity = 0
    
    for row in indices:
        for col in indices:
            if row != col:
                avg_similarity += corr[row][col] / count_not_identity
    return avg_similarity

In [65]:
similarity_scores = [calculate_similarity_score(song['chorus']) for song in songs]

In [75]:
songs = [song_info | {'repetitiveness': repetitiveness} for song_info, repetitiveness in zip(songs, similarity_scores)]

In [76]:
songs

[{'song': 'Blinding Lights by The Weeknd',
  'artist': 'The Weeknd',
  'chorus': ["I said, ooh, I'm blinded by the lights",
   "No, I can't sleep until I feel your touch",
   "I said, ooh, I'm drowning in the night",
   "Oh, when I'm like this, you're the one I trust",
   'Hey, hey, hey'],
  'repetitiveness': 0.26507532447576526},
 {'song': 'Radioactive by Imagine Dragons',
  'artist': 'Imagine Dragons',
  'chorus': ['Whoa-oh, whoa',
   "I'm radioactive, radioactive",
   'Whoa-oh, whoa',
   "I'm radioactive, radioactive"],
  'repetitiveness': 0.4640672504901885},
 {'song': 'Sail by AWOLNATION',
  'artist': 'AWOLNATION',
  'chorus': ['Sail', 'Sail', 'Sail', 'Sail', 'Sail'],
  'repetitiveness': 1.0000000000000002},
 {'song': 'How Do I Live by LeAnn Rimes',
  'artist': 'LeAnn Rimes',
  'chorus': ['How do I live without you? I want to know',
   'How do I breathe without you if you ever go?',
   'How do I ever, ever survive?',
   'How do I, how do I, oh, how do I live?'],
  'repetitiveness'

In [79]:
with open('data/repetition.json', 'x') as json_file:
    json.dump(songs, json_file)

In [81]:
with open('data/songs_grouped_by_genre.json', 'r') as json_file:
    songs_by_genre = json.load(json_file)

In [86]:
songs_by_genre = dict(zip(songs_by_genre.keys(), map(set, songs_by_genre.values())))

In [87]:
songs_by_genre

{'pop': {"'Tis The Damn Season by Taylor Swift",
  "(There's No Place Like) Home For The Holidays by Perry Como",
  '1 Step Forward, 3 Steps Back by Olivia Rodrigo',
  '10,000 Hours by Dan + Shay & Justin Bieber',
  '2 Much by Justin Bieber',
  '24 by Kanye West',
  '24 by Money Man Featuring Lil Baby',
  '34+35 by Ariana Grande',
  '34+35 by Ariana Grande Feat. Doja Cat & Megan Thee Stallion',
  '7 Rings by Ariana Grande',
  '8 by Billie Eilish',
  'A-O-K by Tai Verdes',
  'AP by Pop Smoke',
  'Adore You by Harry Styles',
  'After Hours by The Weeknd',
  'Afterglow by Ed Sheeran',
  'Afterglow by Taylor Swift',
  'Alice by Lady Gaga',
  'All I Know So Far by P!nk',
  'All I Want For Christmas Is You by Mariah Carey',
  'All I Want by Olivia Rodrigo',
  'All The Good Girls Go To Hell by Billie Eilish',
  'All To Myself by Dan + Shay',
  'Alone Again by The Weeknd',
  'Always Do by The Kid LAROI',
  'Always Forever by Bryson Tiller',
  'Always Remember Us This Way by Lady Gaga',
  'Am I

In [88]:
repetitiveness_dict = dict(zip(['pop', 'rap', 'rock'], [0,0,0]))

In [89]:
repetitiveness_dict

{'pop': 0, 'rap': 0, 'rock': 0}

In [91]:
pop_counter = rap_counter = rock_counter = 0

for song in songs:
    if song['song'] in songs_by_genre['pop']:
        repetitiveness_dict['pop'] += song['repetitiveness']
        pop_counter += 1
    elif song['song'] in songs_by_genre['rap']:
        repetitiveness_dict['rap'] += song['repetitiveness']
        rap_counter += 1
    elif song['song'] in songs_by_genre['rock']:
        repetitiveness_dict['rock'] += song['repetitiveness']
        rock_counter += 1

In [93]:
repetitiveness_dict['pop'] /= pop_counter
repetitiveness_dict['rap'] /= rap_counter
repetitiveness_dict['rock'] /= rock_counter

In [94]:
repetitiveness_dict

{'pop': 0.2967372499160073,
 'rap': 0.2665232064116793,
 'rock': 0.2659479192314834}